# UPLOAD TABLEs TO SF

In [2]:
#!pip install --upgrade snowflake-connector-python
import snowflake.connector
import pandas as pd
import numpy as np
from snowflake.connector.pandas_tools import write_pandas  ## TO UPLOAD DATA INTO SF

C:\Users\VYesilbas\AppData\Roaming\Python\Python38\site-packages\snowflake\connector\options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (7.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


You may receive the below error messages. Just install them
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [1]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
ctx = snowflake.connector.connect(
    user = "VYESILBAS@HORIZONMEDIA.COM",  ## Change your username
    account = "hmimtd.us-east-1",    ## 
    authenticator= "externalbrowser",
    warehouse = "MARKETING_SCIENCE_WH",
    database = "HORIZON_PROD",
   schema = "ADVANCEDANALYTICS_SANDBOX")  ## This is needed when uploading a data\frame into SF Table
cs = ctx.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [19]:
#READ THE FILE AND SAVE IT AS A DATAFRAME --- CHANGE THE FILENAME
file_to_uploadSF_df = pd.read_csv("census_pop_2019.csv", low_memory = False)

In [20]:
#check the file-datarframe that will be uploaded to SF
file_to_uploadSF_df.head()

,zipcode,Total_baa,Above18_baa,Total_native,Above18_native,Total_asian,Above18_asian,Total_pacific,Above18_pacific,Total_other,...,Total_Above18,Total,pct_Above18_baa,pct_Above18_asian,pct_Above18_native,pct_Above18_two_or_more_race,pct_Above18_pacific,pct_Above18_other,pct_Above18_white,pct_Above18_hispanic
0,601,182,135,32,32,2,2,0,0,4045,...,16862,21602,0.008006,0.000119,0.001898,0.009014,0.0,0.177678,0.003143,0.800142
1,602,908,791,46,28,0,0,0,0,1194,...,35111,43020,0.022529,0.000000,0.000797,0.077981,0.0,0.023525,0.066503,0.808664
2,603,1514,1306,68,68,108,101,11,0,7342,...,45524,56352,0.028688,0.002219,0.001494,0.013268,0.0,0.128965,0.012389,0.812978
3,606,92,92,17,17,0,0,0,0,3299,...,7988,9874,0.011517,0.000000,0.002128,0.009264,0.0,0.325363,0.006760,0.644967
4,610,461,461,0,0,0,0,1,0,2095,...,25399,31185,0.018150,0.000000,0.000000,0.072168,0.0,0.060239,0.039214,0.810229


In [26]:
#file_to_uploadSF_df ["zipcode"] = file_to_uploadSF_df.zipcode.to_string
file_to_uploadSF_df ["zipcode"] = file_to_uploadSF_df ["zipcode"].apply(str)

In [27]:
file_to_uploadSF_df.dtypes

zipcode                          object
Total_baa                         int64
Above18_baa                       int64
Total_native                      int64
Above18_native                    int64
Total_asian                       int64
Above18_asian                     int64
Total_pacific                     int64
Above18_pacific                   int64
Total_other                       int64
Above18_other                     int64
Total_two_or_more_race            int64
Above18_two_or_more_race          int64
Total_white                       int64
Above18_white                     int64
Total_hispanic                    int64
Above18_hispanic                  int64
Total_Above18                     int64
Total                             int64
pct_Above18_baa                 float64
pct_Above18_asian               float64
pct_Above18_native              float64
pct_Above18_two_or_more_race    float64
pct_Above18_pacific             float64
pct_Above18_other               float64


In [28]:
file_to_uploadSF_df["zipcode"]=file_to_uploadSF_df["zipcode"].apply(lambda x: x.zfill(5))

In [45]:
sql_query = 'CREATE or replace TABLE "HORIZON_PROD"."ADVANCEDANALYTICS_SANDBOX"."CENSUS_2019_ABOVE_18_YEARS_OF_AGE" \
("zipcode"                          VARCHAR(5), \
"Total_baa"                         INTEGER, \
"Above18_baa"                       INTEGER, \
"Total_native"                      INTEGER, \
"Above18_native"                    INTEGER, \
"Total_asian"                       INTEGER, \
"Above18_asian"                     INTEGER, \
"Total_pacific"                     INTEGER, \
"Above18_pacific"                   INTEGER, \
"Total_other"                       INTEGER, \
"Above18_other"                     INTEGER, \
"Total_two_or_more_race"            INTEGER, \
"Above18_two_or_more_race"          INTEGER, \
"Total_white"                       INTEGER, \
"Above18_white"                     INTEGER, \
"Total_hispanic"                    INTEGER, \
"Above18_hispanic"                  INTEGER, \
"Total_Above18"                     INTEGER, \
"Total"                             INTEGER, \
"pct_Above18_baa"                 FLOAT, \
"pct_Above18_asian"               FLOAT, \
"pct_Above18_native"              FLOAT, \
"pct_Above18_two_or_more_race"    FLOAT, \
"pct_Above18_pacific"             FLOAT, \
"pct_Above18_other"               FLOAT, \
"pct_Above18_white"               FLOAT, \
"pct_Above18_hispanic"            FLOAT)'
cs.execute(sql_query)

In [40]:
#file_to_uploadSF_df.isnull().any()

In [5]:
# Row and column numbers of the file - dataframe
file_to_uploadSF_df.shape

(44, 2)

In [17]:
# Create Table Within SF -- -- CHANGE THE TABLE_NAME
sql_query = 'CREATE or replace TABLE "HORIZON_PROD"."ADVANCEDANALYTICS_SANDBOX"."TABLE_NAME_IN_SF" ("KEYWORD" VARCHAR (100), "COUNT" INTEGER)'
cs.execute(sql_query)

In [46]:
# Load PANDAS DataFRAME into a SnoWflake table -- CHANGE THE TABLE_NAME
success, nchunks, nrows, _ = write_pandas(ctx, file_to_uploadSF_df, 'CENSUS_2019_ABOVE_18_YEARS_OF_AGE')